### Decorators (Part 1)

Вспомните пример из предыдущего раздела, где мы написали простое замыкание для подсчета количества запусков функции:

In [2]:
def counter(fn):
    count = 0

    def inner(*args, **kwargs):
        nonlocal count
        count += 1
        print('Function {0} was called {1} times'.format(fn.__name__, count))
        return fn(*args, **kwargs)
    return inner

In [3]:
def add(a, b=0):
    """
    returns the sum of a and b
    """
    return a + b

In [4]:
help(add)

Help on function add in module __main__:

add(a, b=0)
    returns the sum of a and b



Вот адрес памяти, на который указывает `add`:

In [5]:
id(add)

140440501743232

Теперь создадим замыкание, используя функцию `add` в качестве аргумента функции `counter`:

In [6]:
add = counter(add)

И вы заметите, что `add` больше не является той же функцией, что и раньше. Действительно, адрес памяти, на который указывает `add`, больше не тот же:

In [7]:
id(add)

140440501743552

In [8]:
add(1, 2)

Function add was called 1 times


3

In [9]:
add(2, 2)

Function add was called 2 times


4

Произошло то, что мы поместили нашу функцию **add** «через» функцию **counter** — мы обычно говорим, что мы **декорировали** нашу функцию **add**.

И мы называем эту функцию **counter** **декоратором**.

Существует сокращенный способ декорирования нашей функции без необходимости ввода:

``func = counter(func)``

In [10]:
@counter
def mult(a: float, b: float=1, c: float=1) -> float:
    """
    returns the product of a, b, and c
    """
    return a * b * c

In [11]:
mult(1, 2, 3)

Function mult was called 1 times


6

In [12]:
mult(2, 2, 2)

Function mult was called 2 times


8

Давайте немного поразмышляем над нашими двумя декорированными функциями:

In [13]:
add.__name__

'inner'

In [14]:
mult.__name__

'inner'

Как видите, имя функции больше не **add** или **mult**, а вместо этого это имя **внутренней** функции в нашем декораторе.

In [15]:
help(add)

Help on function inner in module __main__:

inner(*args, **kwargs)



In [16]:
help(mult)

Help on function inner in module __main__:

inner(*args, **kwargs)



Как видите, мы также потеряли наши строки документации и аннотации параметров!

А как насчет анализа параметров **add** и **mult**:

In [17]:
import inspect

In [18]:
inspect.getsource(add)

"    def inner(*args, **kwargs):\n        nonlocal count\n        count += 1\n        print('Function {0} was called {1} times'.format(fn.__name__, count))\n        return fn(*args, **kwargs)\n"

In [19]:
inspect.getsource(mult)

"    def inner(*args, **kwargs):\n        nonlocal count\n        count += 1\n        print('Function {0} was called {1} times'.format(fn.__name__, count))\n        return fn(*args, **kwargs)\n"

Даже подпись исчезла:

In [19]:
inspect.signature(add)

<Signature (*args, **kwargs)>

In [20]:
inspect.signature(mult)

<Signature (*args, **kwargs)>

Даже документация по параметрам по умолчанию исчезла:

In [21]:
inspect.signature(add).parameters

mappingproxy({'args': <Parameter "*args">, 'kwargs': <Parameter "**kwargs">})

В общем случае, когда мы создаем декорированные функции, мы в конечном итоге «теряем» большую часть метаданных нашей исходной функции!

Однако мы **можем** вернуть эту информацию обратно — это может быть довольно сложно.

Давайте посмотрим, как мы можем сделать это для некоторых простых вещей, таких как строка документации и имя функции.

In [22]:
def counter(fn):
    count = 0

    def inner(*args, **kwargs):
        nonlocal count
        count += 1
        print("{0} was called {1} times".format(fn.__name__, count))
    inner.__name__ = fn.__name__
    inner.__doc__ = fn.__doc__
    return inner

In [23]:
@counter
def add(a: int, b: int=10) -> int:
    """
    returns sum of two integers
    """
    return a + b

In [24]:
help(add)

Help on function add in module __main__:

add(*args, **kwargs)
    returns sum of two integers



In [25]:
add.__name__

'add'

По крайней мере, у нас есть docstring и имя функции... Но что насчет параметров? Наша настоящая функция **add** принимает два позиционных параметра, но поскольку замыкание использовало общий способ принятия **\*args** и **\*\*kwargs**, мы теряем эту информацию

Мы можем использовать специальную функцию в модуле **functools**, называемую **wraps**. Фактически, эта функция сама по себе является декоратором!

In [26]:
from functools import wraps

In [27]:
def counter(fn):
    count = 0

    @wraps(fn)
    def inner(*args, **kwargs):
        nonlocal count
        count += 1
        print("{0} was called {1} times".format(fn.__name__, count))
        return fn(*args, **kwargs)

    return inner

In [28]:
@counter
def add(a: int, b: int=10) -> int:
    """
    returns sum of two integers
    """
    return a + b

In [29]:
help(add)

Help on function add in module __main__:

add(a:int, b:int=10) -> int
    returns sum of two integers



Ура!!! Всё вернулось на круги своя.

In [30]:
inspect.getsource(add)

'@counter\ndef add(a: int, b: int=10) -> int:\n    """\n    returns sum of two integers\n    """\n    return a + b\n'

In [31]:
inspect.signature(add)

<Signature (a:int, b:int=10) -> int>

In [32]:
inspect.signature(add).parameters

mappingproxy({'a': <Parameter "a:int">, 'b': <Parameter "b:int=10">})

---